## Spark Project
### Predicting ow much a customer will spend yearly .
#### PySpark ML 
@Author - Amruta Abhyankar

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv("/Users/amrutaabhyankar/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv",
              inferSchema=True, header= True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [7]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol='features')

In [11]:
# Transforming the data
output = assembler.transform(data)

In [12]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [14]:
final_data = output.select('features','Yearly Amount Spent')

In [15]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [16]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                342|
|   mean|  501.1819156576027|
| stddev|  77.09389206419159|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                158|
|   mean|  495.2709112303496|
| stddev|  84.03168058742558|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [18]:
lr = LinearRegression(featuresCol="features",
                            labelCol="Yearly Amount Spent")

In [19]:
lr_model = lr.fit(train_data)

In [20]:
test_results = lr_model.evaluate(test_data)

In [21]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 12.282042151134192|
| -10.57544238864773|
|  5.011407502244424|
| -8.205863285245641|
| 18.851096655093045|
|  5.127926226104023|
| 2.0032830498202543|
|-14.548798991705326|
|   17.1078973825268|
| -5.667400888496161|
|-26.299116491273367|
|  -9.80427869613186|
|-0.9630420539276656|
|-0.2835517980246891|
|   9.04571437921203|
|-2.3429266941016067|
|-11.561683592092436|
| 13.204237435078426|
| -4.600621888301816|
|  1.019936286458858|
+-------------------+
only showing top 20 rows



In [22]:
test_results.rootMeanSquaredError

10.395669881479254

In [23]:
test_results.r2

0.9845980291997689

In [24]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [25]:
unlabled_data = test_data.select("features")

In [26]:
unlabled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[31.0472221394875...|
|[31.1280900496166...|
|[31.3123495994443...|
|[31.3662121671876...|
|[31.4459724827577...|
|[31.5741380228732...|
|[31.6098395733896...|
|[31.6253601348306...|
|[31.6739155032749...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.8530748017465...|
|[31.8648325480987...|
|[31.9096268275227...|
|[31.9673209478824...|
|[32.0047530203648...|
+--------------------+
only showing top 20 rows



In [27]:
predictions = lr_model.transform(unlabled_data)

In [28]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 396.3583089214933|
|[30.3931845423455...|330.50431219184134|
|[31.0472221394875...|387.48599168677697|
|[31.1280900496166...| 565.4585500323003|
|[31.3123495994443...| 444.7403213728476|
|[31.3662121671876...| 425.4609563303809|
|[31.4459724827577...| 482.8736818853083|
|[31.5741380228732...| 558.9580711522922|
|[31.6098395733896...|427.43765226858136|
|[31.6253601348306...|382.00430164542036|
|[31.6739155032749...|502.02418440115457|
|[31.8093003166791...|  546.576178058973|
|[31.8124825597242...| 393.7733870377249|
|[31.8186165667690...|446.70222516816034|
|[31.8209982016720...| 415.6295666340013|
|[31.8530748017465...| 461.6280501564536|
|[31.8648325480987...| 451.4529640689061|
|[31.9096268275227...| 550.2417982381608|
|[31.9673209478824...| 450.3504631279541|
|[32.0047530203648...| 462.7260448341706|
+--------------------+------------